In [175]:
import pandas as pd
import string
import numpy as np

In [176]:
df=pd.read_csv("test.csv",encoding="ISO-8859-1")
df=df[['text','sentiment']]
df=df.dropna()


In [177]:
df.text=df.text.str.lower().str.translate(str.maketrans('', '', string.punctuation))


In [178]:
df

,text,sentiment
0,last session of the day httptwitpiccom67ezh,neutral
1,shanghai is also really exciting precisely s...,positive
2,recession hit veronique branquinho she has to ...,negative
3,happy bday,positive
4,httptwitpiccom4w75p i like it,positive
...,...,...
3529,its at 3 am im very tired but i cant sleep bu...,negative
3530,all alone in this old house again thanks for ...,positive
3531,i know what you mean my little dog is sinking...,negative
3532,sutra what is your next youtube video gonna be...,positive


In [179]:
df=df.sample(frac=1)


In [180]:
for i in df.loc[df.sentiment[df.sentiment=='positive'].iloc[:5].index, 'text']:
    print(i)
    print("----------")


didnt meet rosie o donnell but it was still a fun night
----------
going to al ain  need to check out one store hope i find things that im looking for
----------
just got back from kuantan had sooooooo much fun
----------
 always welcome sweetie hugs
----------
 rofl u did  hahaha lemme tell salman munir p p
----------


In [181]:
for i in df.loc[df.sentiment[df.sentiment=='negative'].iloc[:5].index, 'text']:
    print(i)
    print("----------")


twitter fam im about to hop back on aim i went ghost on yall lolz sowwy
----------
i have been playing skate for two hours now i need to get out and actually skate but its too late
----------
bebop thats lame sorry
----------
 oh nosorry to hear that 
----------
  nooo  i dont know whyi click on tweetdeck025manualblink182air and it doesnt run so sad
----------


In [182]:
for i in df.loc[df.sentiment[df.sentiment=='neutral'].iloc[:5].index, 'text']:
    print(i)
    print("----------")


watching the notebook
----------
nooooooooanother headachehopefully not a migraine
----------
 jia you 
----------
 httptwitpiccom1aida  snood
----------
tdh can you install ruby gems and rails on keep please
----------


In [183]:
training_df=df.iloc[:int(len(df)*0.7)]
testing_df=df.iloc[int(len(df)*0.7):]


In [184]:
training_df

,text,sentiment
3498,didnt meet rosie o donnell but it was still a ...,positive
642,twitter fam im about to hop back on aim i went...,negative
1043,watching the notebook,neutral
2351,i have been playing skate for two hours now i ...,negative
778,going to al ain need to check out one store h...,positive
...,...,...
2455,its a bad start to the day when youre sweating...,negative
276,thinks revising in the garden from 8 this morn...,negative
527,epicentre wheelock place tourists only,neutral
1711,same to you bud kudos on your hardcore gym tr...,positive


In [185]:
testing_df

,text,sentiment
946,you said darling what happened did i fall asle...,positive
1320,httptwitpiccom67s14 saw a family of five get ...,neutral
245,follow me too,neutral
3252,the problem is superstars always deliver huge...,negative
1367,is watching mock the week and is slowing cooking,neutral
...,...,...
3046,back home after a funeral for my wifes grandfa...,neutral
3521,munchin bacon butties woohoo my faves,positive
3153,thanks mr dj,positive
2443,good grief i cant say much i was driving home...,neutral


In [186]:
fraction_neutral = (training_df['sentiment']=='neutral').mean()
fraction_positive=( training_df['sentiment']=='positive').mean()
fraction_negative= (testing_df['sentiment']=='negative').mean()


In [187]:
train_positive_words=''.join(training_df[training_df['sentiment']=='positive'].text).split(' ')
train_negative_words=''.join(training_df[training_df['sentiment']=='negative'].text).split(' ')
train_neutral_words=''.join(training_df[training_df['sentiment']=='neutral'].text).split(' ')



In [188]:
common_words_pn=set(train_positive_words).intersection(set(train_negative_words))
common_words=set(common_words_pn).intersection(set(train_neutral_words))

In [189]:
train_neutral_bow=dict()
for w in common_words:
    train_neutral_bow[w]=train_neutral_words.count(w)/len(train_neutral_words)
    
train_positive_bow=dict()
for w in common_words:
    train_positive_bow[w]=train_positive_words.count(w)/len(train_positive_words)
train_negative_bow=dict()
for w in common_words:
    train_negative_bow[w]=train_negative_words.count(w)/len(train_negative_words)


In [257]:
for key, value in list(train_positive_bow.items())[:10]:
    print(f'{key}: {value}')

: 0.055175318521638414
pretty: 0.000737074865747078
please: 0.00031588922817731913
rained: 0.00021059281878487945
him: 0.0006317784563546383
seriously: 0.00010529640939243972
burrito: 0.00010529640939243972
loads: 0.00010529640939243972
youre: 0.001158260503316837
done: 0.0005264820469621986


In [258]:
for key, value in list(train_negative_bow.items())[:10]:
    print(f'{key}: {value}')

: 0.04207689916053477
pretty: 0.00020727536532283138
please: 0.00041455073064566275
rained: 0.00010363768266141569
him: 0.001347289874598404
seriously: 0.00010363768266141569
burrito: 0.00010363768266141569
loads: 0.00010363768266141569
youre: 0.0006218260959684941
done: 0.0005181884133070785


# Classification

Predict on Test Set P(positive|"awww holly shes so cute")∝ P("awww holly shes so cute "|Positive)*P(positive)

we make IID assumption on the words so it can be written as

P("aww"|positive).P("holly"|positive)...................P("cute"|postive)*P(postive)




# Predicition

In [259]:
t='she is so cute'
valid=(w for w in t if w in train_positive_words)
print(valid)

<generator object <genexpr> at 0x000002228B0DCD40>


In [260]:
def predict(t,verbose=False):
    valid_words = [w for w in t.split() if w in train_positive_bow]

    positive_prob=[train_positive_bow[w] for w in valid_words]
    negative_prob=[train_negative_bow[w] for w in valid_words]
    neutral_prob=[train_neutral_bow[w] for w in valid_words]

    
    if verbose:
        df=pd.DataFrame()
        df['words']=valid_words
        df['positive_prob']=positive_prob
        df['negative_prob']=negative_prob
        df['neutral_prob']=neutral_prob

        df['ratio_1']=[s/n if n>0 else np.inf for s,n in zip(positive_prob,negative_prob)]
        df['ratio_2']=[s/n if n>0 else np.inf for s,n in zip(positive_prob,neutral_prob)]
        df['ratio_3']=[s/n if n>0 else np.inf for s,n in zip(negative_prob,neutral_prob)]
                
        print(df)
    
    pos_score=sum([np.log(p) for p in positive_prob])+np.log(fraction_positive)
    
    neg_score=sum([np.log(p) for p in negative_prob])+np.log(fraction_negative)

    neutral_score=sum([np.log(p) for p in neutral_prob])+np.log(fraction_neutral)

    if verbose:
        print("positive score is :",pos_score)
        print("Negative score is :",neg_score)
        print('neutral Score is :',neutral_score)
    if pos_score>=neg_score and pos_score>=neutral_score:
        return "positive"
    if neg_score>=pos_score and neg_score>=neutral_score:
        return "negative"
    if neutral_score>=neg_score and pos_score<=neutral_score:
        return "neutral"

In [261]:
predict("she is not so  cute",True)

  words  positive_prob  negative_prob  neutral_prob   ratio_1   ratio_2  \
0   she       0.001790       0.001140      0.001156  1.570190  1.548000   
1    is       0.008529       0.010571      0.009416  0.806828  0.905796   
2   not       0.002843       0.006322      0.005947  0.449707  0.478059   
3    so       0.007687       0.009224      0.005451  0.833352  1.410032   
4  cute       0.000737       0.000104      0.000165  7.112035  4.461883   

    ratio_3  
0  0.985868  
1  1.122663  
2  1.063045  
3  1.692000  
4  0.627371  
positive score is : -30.230909318666672
Negative score is : -31.622925249125974
neutral Score is : -31.38442162479136


'positive'

# The above is not correctly classified cause we have taken IID assumption so although there is not present in the sentence but since the effect of positive prob of other words is greater which makes the total prob positive

In [262]:
predict("she is  so  cute")

'positive'

In [263]:
predict("lets meet, its gonna be crazy")

'neutral'

In [264]:
predictions=testing_df.text.apply(lambda t: predict(t))
predictions

946      neutral
1320     neutral
245      neutral
3252    negative
1367     neutral
          ...   
3046     neutral
3521     neutral
3153    positive
2443    positive
1099     neutral
Name: text, Length: 1061, dtype: object

In [265]:
predictions=testing_df.text.apply(lambda t: predict(t))
print(np.sum(testing_df['sentiment']=='positive'))
t=np.sum((predictions == 'positive') & (testing_df['sentiment'] == 'positive'))
print(t)
recall = np.sum((predictions == 'postive') & (testing_df['sentiment'] == 'positive')) / np.sum(testing_df['sentiment'] == 'positive')
print('Fraction positive Correctly Detected: %s'%recall)


356
208
Fraction positive Correctly Detected: 0.0


In [266]:
# Assuming predictions and true_sentiments are Pandas Series or NumPy arrays
recall_positive = np.sum((predictions == 'positive') & (testing_df['sentiment'] == 'positive')) / np.sum(testing_df['sentiment']== 'positive')
recall_negative = np.sum((predictions == 'negative') & (testing_df['sentiment']=='negative')) / np.sum(testing_df['sentiment'] == 'negative')
recall_neutral = np.sum((predictions == 'neutral') & (testing_df['sentiment']== 'neutral')) / np.sum(testing_df['sentiment']== 'neutral')

print('Recall for Positive Sentiment:', recall_positive)
print('Recall for Negative Sentiment:', recall_negative)
print('Recall for Neutral Sentiment:', recall_neutral)


Recall for Positive Sentiment: 0.5842696629213483
Recall for Negative Sentiment: 0.44981412639405205
Recall for Neutral Sentiment: 0.5642201834862385
